In [ ]:
import os 
es = "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.1"
http_scala='org.apache.httpcomponents:httpclient:4.5.13'
common_client='commons-httpclient:commons-httpclient:3.1'

os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{},{} pyspark-shell".format(es, http_scala,common_client))
from pyspark.sql import SparkSession
from elasticsearch import Elasticsearch, helpers


In [ ]:
spark = SparkSession.builder\
    .appName('hdfs-to-elasticsearch')\
    .master('spark://bacener:7077').getOrCreate()

In [ ]:
df = spark.read.parquet('hdfs://0.0.0.0:9000/user/tuannha/clean-data')

In [ ]:
df.printSchema()

In [ ]:
data = df.toPandas()

In [ ]:
def convert_data(data):
    data['price'] = data['price'].astype(float)/100000
    data['price_before_discount'] = data['price_before_discount'].astype(float)/100000
    data['rating_star'] = data.item_rating.apply(lambda x: eval(x)['rating_star'])
    data['category']= data.categories.apply(lambda x: eval(x)[0]['display_name'])
    data.drop(columns=['item_rating', 'categories'],inplace=True)
    cols = ['historical_sold','raw_discount','cmt_count','price','liked_count','rating_star']
    data[cols] = data[cols].astype(float)
    return data

df2 = convert_data(data)
df2

In [ ]:
final_df = spark.createDataFrame(df2)

In [ ]:
final_df.printSchema()

In [ ]:
final_df.show(5)

In [ ]:
final_df.write.format(
    "org.elasticsearch.spark.sql"
).option(
    "es.resource", '%s/%s' % ('item_shopee_v1', 'data')
).option(
    "es.nodes", 'localhost'
).option(
    "es.port", 9200
).save()